一般来说，时间序列预测通常都面临着训练数据不足的问题：

1. 金融数据的演变模式是非常复杂的，想要建模这种复杂模式，我们客观上需要更加复杂的模型

2. 随着模型的复杂化，模型参数的数量也大幅增长。如果训练集数据量没有同步扩大，就会陷入过拟合问题，模型可以轻松记住所有的训练集数据

3. 如果横向扩展数据量，吸收不同的序列，那么就要承担序列背后标的不同带来的损失，因为这样的模型只能建模共享的规律，每个资产单独的规律反而变成了噪声

4. 如果纵向扩展数据量，用同一个时间序列更久的历史数据，又要面临时间序列模式迁移的问题

5. 对于计算机视觉，可以通过数据增强的方式，变换原始数据（如旋转、裁剪）、添加噪声或组合生成新样本来丰富数据集

6. 在部分论文中，通过模拟ARIMA过程人工生成一些时间序列数据，但是这些数据是现实中不存在的，且金融数据有OHLCV等丰富结构，人工生成的数据几乎可以肯定会破坏真实数据的效果

7. 在RNN模型中，由于RNN可以接受变长序列，我提出了一个随机裁剪的方法，每次随机裁去一部分前端序列 (modules.truncate.SequenceTruncate) 来让模型每次都看到稍微不一样一些的序列。但是这种办法只能缓解对久远历史的过拟合，且不适用于定长输入模块。

在有了一个效果良好的assets_embedding 模块之后，我们就可以解决时间序列训练数据不足的问题。我们可以随机生成一些合理的组合资产，并且将组合资产的历史收益率和其他线性可加的特征加入训练集。基资产具有一个向量表示，表示了其在多维资产空间中的风险暴露倾向而这个embedding也是线性可加的。transformer的注意力机制可以通过这个embedding读取到资产之间的相关关系，也可以读取到组合资产的属性构成。

相比于人工生成的时间序列，这些组合资产时间序列是有意义的，因为他们是可交易的，真实存在的一种资产，这是这种组合时间序列的优势；

组合资产与基资产的区别主要是其价格信息并不像基资产那样被交易者持续关注。交易信息会影响交易者行为，交易者的关注度反过来也会影响交易信息，这是市场混沌系统的重要成因。举例来说就是基资产到达支撑位的时候可能会有支撑效应，但组合资产就不一定，因为不会有人根据一个随机的组合资产达到了支撑位，就根据这个来改变自己的投资决策，甚至绝大多数交易者都不会意识到这个组合资产的存在。只有一些特殊的组合资产，例如比较重要的套利关系，才会被交易者关注。因此组合资产还是有一定的特殊性，并不完全等同于基资产，尤其是技术面的因素。

因此，在数据集中我们掺杂着基资产和组合资产进行训练，每次输入一个小批量，然后根据这个小批量随机生成一些组合资产扩展这个批量，保证即便扩展的组合资产的效果不好（比如生成了一个玉米+橡胶+燃料油为主的这种无厘头组合）注意力机制也可以关注到原本的基资产，至少效果不会变差。

假设我们的输入结构为：(batch_size, num_assets, seq_len, feature_dim) 输出结构为 (batch_size, num_assets, label_dim)

定义超参数 expand_dim 该模块会随机生成一个大小为(batch_size, expand_dim, num_assets) 的矩阵，其第三个维度的绝对值之和恒等于1，保证资产组合的有效性

使用batch矩阵乘法，得到(batch_size, expand_dim, seq_len, feature_dim) 和 (batch_size, expand_dim, label_dim) 两个扩展的组合资产信息

最后concat到原序列上输出(batch_size, num_assets + expand_dim, seq_len, feature_dim) 输出结构为 (batch_size, num_assets + expand_dim, label_dim)

In [1]:
import torch
import torch.nn as nn

batch_size = 128
num_assets = 46
expand_dim = 20

# 生成随机的无杠杆的多空持仓
weights = 2 * torch.rand(batch_size, expand_dim, num_assets) - 1 
# 计算持仓绝对值
l1_norms = torch.sum(torch.abs(weights), dim=-1, keepdim=True)
# 归一化持仓，保证每个资产的绝对仓位的和 = 1
normalized_weights = weights / (l1_norms)
print(normalized_weights.shape)

torch.Size([128, 20, 46])


还有一点需要注意的是，我们需要保证原始的特征维度必须是线性可加的；例如最高价和最低价就不是线性可加的，因为两个资产的最高价并不一定出现在同时刻，因此这个资产组合的最高价并不等于两个资产的最高价的线性组合。

当然，如果有足够的数据可以扩展到分钟线去处理，那也就能提取这个日线的组合最高价信息了

In [12]:
class PortfolioExpand(nn.Module):
    """
    创建资产组合来增强数据，这些组合是原始资产的线性组合，权重被约束为 L1 范数等于 1，以模拟一个全额投资的（多空）投资组合。
    """
    def __init__(self):
        super().__init__()


    def _portfolio_weights(self,batch_size, num_base_assets, num_full_assets,  device) :
        """生成资产组合矩阵，在最后一个维度上绝对值之和等于1"""
        weights = 2 * torch.rand(size = (batch_size, num_full_assets - num_base_assets, num_base_assets), device=device) - 1
        l1_norms = torch.sum(torch.abs(weights), dim=-1, keepdim=True)
        normalized_weights = weights / (l1_norms + 1e-16)
        return normalized_weights

    def forward(self, x, embed ,y, num_full_assets):
        """
        输入:
            - x: (batch_size, num_base_assets, seq_len, feature_dim)
            - y: (batch_size, num_base_assets, label_dim)
            - embed: (batch_size, num_base_assets, num_embedding)
        输出:
            - expanded_x: (batch_size, num_full_assets, seq_len, feature_dim)
            - expanded_y: (batch_size, num_full_assets, label_dim)
            - expanded_embed: (batch_size, num_full_assets, num_embedding)
        """
        # 验证输入维度
        if x.dim() != 4 or y.dim() != 3:
            raise ValueError(f"输入维度错误! x应为4维, y应为3维, "
                             f"但实际为 x: {x.dim()}维, y: {y.dim()}维")
        if x.shape[0] != y.shape[0] or x.shape[1] != y.shape[1]:
            raise ValueError("x 和 y 的 batch_size 和 num_assets 维度必须匹配")

        batch_size, num_base_assets, seq_len, feature_dim = x.shape
        device = x.device
        weights = self._portfolio_weights(batch_size, num_base_assets, num_full_assets, device)

        # torch.einsum 可以在第二个维度上进行矩阵乘法（也可以用bmm）
        expanded_x = torch.einsum('ben,bnsf->besf', weights, x)
        expanded_y = torch.einsum('ben,bnl->bel', weights, y)
        expanded_embed = torch.einsum('ben,bnl->bel', weights, embed)
        
        # 拼接基资产和组合资产
        output_x = torch.cat([x, expanded_x], dim=1)
        output_y = torch.cat([y, expanded_y], dim=1)
        output_embed = torch.cat([embed, expanded_embed], dim=1)

        return output_x, output_embed, output_y


if __name__ == '__main__':
    device = 'cuda:0'
    portfolio_generator = PortfolioExpand()
    input_x = torch.randn(size = (128, 54, 90, 10), device=device)
    input_y = torch.randn(size = (128, 54, 4), device=device)
    input_embed = torch.randn(size = (128, 54, 10), device=device)
    output_x, output_embed, output_y = portfolio_generator(input_x, input_embed, input_y, 100)
    print(f"       x shape: {input_x.shape}")
    print(f"expand x shape: {output_x.shape}")
    print(f"       y shape: {input_y.shape}")
    print(f"expand y shape: {output_y.shape}")
    print(f"       embed shape: {input_embed.shape}")
    print(f"expand embed shape: {output_embed.shape}")

       x shape: torch.Size([128, 54, 90, 10])
expand x shape: torch.Size([128, 100, 90, 10])
       y shape: torch.Size([128, 54, 4])
expand y shape: torch.Size([128, 100, 4])
       embed shape: torch.Size([128, 54, 10])
expand embed shape: torch.Size([128, 100, 10])
